<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/Customer_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import StackingClassifier


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
trainfile = r'/content/Cell Phone Churn-TRAIN.csv'
train_data = pd.read_csv(trainfile)
testfile = r'/content/Cell Phone Churn-TEST.csv'

test_data = pd.read_csv(testfile)
print(train_data.shape)
print(train_data.head())
print()
print(test_data.shape)
print(test_data.head())

(1695, 24)
   MOUMO  MOUPMO  MOU3MO  MOUCH1M  MOUCH3M  CUM3MCH  CUSTMOS  LONGD  CALLW  \
0    105      15      99       90      -84        6       11      0      0   
1    999    1200    1145     -201       55     -146        3      1      1   
2      5       2       1        3        1        4       23      0      0   
3    600     456     398      144       58      202       13      0      1   
4      2       0       0        0        0        0        1      0      0   

   LINES  ...  INCOME  SPORTS NATURE ARTS  HRS_TV         TRAVEL        EDUC  \
0      2  ...   51000       B      1    N      10            NaN   Bachelors   
1      2  ...   80000       A      0    N      40       Domestic     Masters   
2      1  ...   80000       C      0    Y      50  International  HighSchool   
3      1  ...   88000       A      1    N       0  International     Masters   
4      1  ...   29000       B      0    N      75       Domestic  HighSchool   

  TOTMOU  TOTCHNG TARGET  
0    219    

In [ ]:
categoricalFeatures = ["TRAVEL","EDUC", "CONVB","SPORTS", "SEX", 'ARTS']


combined_data = pd.concat([train_data, test_data], keys=[0, 1])

combined_data = pd.get_dummies(combined_data, columns=categoricalFeatures)
print(combined_data['TARGET'])



0  0      0
   1      1
   2      1
   3      1
   4      0
         ..
1  370    0
   371    1
   372    0
   373    1
   374    1
Name: TARGET, Length: 2070, dtype: int64


In [ ]:
#Extract Target Column before doing missing value substitutions and one-hot encoding======
Target_Train_Cols = train_data["TARGET"]#make copy of target column
trainData = train_data.drop(["TARGET"], axis=1) #extracting training data without the target column

In [ ]:
combined_Data = pd.concat([train_data, test_data], keys=[0,1])

combined_Data.isnull().sum().sort_values(ascending=False)

,0
TRAVEL,360
MOUMO,0
MOUPMO,0
TOTCHNG,0
TOTMOU,0
EDUC,0
HRS_TV,0
ARTS,0
NATURE,0
SPORTS,0


In [ ]:
#Define threshold for dropping columns
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
td1=combined_Data.dropna(thresh=percent,axis=1)
print(td1.shape)

1242
(2070, 24)


In [ ]:
#look at what other columns have missing values
td1.isnull().sum().sort_values(ascending=False)

,0
TRAVEL,360
MOUMO,0
MOUPMO,0
TOTCHNG,0
TOTMOU,0
EDUC,0
HRS_TV,0
ARTS,0
NATURE,0
SPORTS,0


In [ ]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS
numeric=td1.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td1[num]=td1[num].fillna(td1[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = td1.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  if(td1.iloc[0][colss]=="N"):
        td1[colss]=td1[colss].fillna("N")
  else:
    td1[colss]=td1[colss].fillna(td1[colss].mode())

print(td1.head(20))


      MOUMO  MOUPMO  MOU3MO  MOUCH1M  MOUCH3M  CUM3MCH  CUSTMOS  LONGD  CALLW  \
0 0     105      15      99       90      -84        6       11      0      0   
  1     999    1200    1145     -201       55     -146        3      1      1   
  2       5       2       1        3        1        4       23      0      0   
  3     600     456     398      144       58      202       13      0      1   
  4       2       0       0        0        0        0        1      0      0   
  5     678    1222     598     -544      624       80       30      1      1   
  6     110      98      56       12       42       54       15      1      0   
  7      97      56      97       41      -41        0        8      0      0   
  8      45      23      66       22      -43      -21       44      0      0   
  9      53      34      29       19        5       24       50      1      1   
  10      1       2       0       -1        0        0        2      0      0   
  11   5000    7000      75 

In [ ]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES
td1.isnull().sum().sort_values(ascending=False)

,0
TRAVEL,360
MOUMO,0
MOUPMO,0
TOTCHNG,0
TOTMOU,0
EDUC,0
HRS_TV,0
ARTS,0
NATURE,0
SPORTS,0


In [ ]:
#DROP COLUMNS THAT STILL HAVE NULL VALUES
print(td1.shape)
td1=td1.drop(columns=["TRAVEL"])
td1.isnull().sum()
print(td1.shape)

(2070, 24)
(2070, 23)


In [ ]:
td1.isnull().sum().sort_values(ascending=False)

,0
MOUMO,0
CONVB,0
TOTCHNG,0
TOTMOU,0
EDUC,0
HRS_TV,0
ARTS,0
NATURE,0
SPORTS,0
INCOME,0


In [ ]:
#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
y_train=X_train["TARGET"]
y_test=X_test["TARGET"]
X_train1=X_train.iloc[:, :-1].copy()
X_test1=X_test.iloc[:, :-1].copy()

In [ ]:

#Separate Train data and test data
X_train = combined_data.xs(0)
X_test = combined_data.xs(1)

print(X_train.shape)
print(X_test.shape)

y_train=X_train["TARGET"]
y_test=X_test["TARGET"]


X_train = X_train.drop('TARGET',axis=1)
X_test = X_test.drop('TARGET',axis=1)

print(X_train.shape)
print(X_test.shape)

(1695, 40)
(375, 40)
(1695, 39)
(375, 39)


In [ ]:
#Decision Tree, Random Forest, Logistic Regression, SVM model result
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(random_state=42)
}

results = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append({
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred)
    })

for result in results:
    print(result)

{'Model': 'Decision Tree', 'Accuracy': 0.9946666666666667, 'Precision': 1.0, 'Recall': 0.9857142857142858, 'F1 Score': 0.9928057553956835}
{'Model': 'Random Forest', 'Accuracy': 0.9946666666666667, 'Precision': 1.0, 'Recall': 0.9857142857142858, 'F1 Score': 0.9928057553956835}
{'Model': 'Logistic Regression', 'Accuracy': 0.9653333333333334, 'Precision': 0.9774436090225563, 'Recall': 0.9285714285714286, 'F1 Score': 0.9523809523809523}
{'Model': 'SVM', 'Accuracy': 0.6506666666666666, 'Precision': 0.6666666666666666, 'Recall': 0.12857142857142856, 'F1 Score': 0.2155688622754491}


In [ ]:
# hyperparameter tuning done for decision tree
param_distributions = {
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 10],
    'criterion': ['gini', 'entropy']
}

decision_tree = DecisionTreeClassifier(random_state=42)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=decision_tree,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='f1',
    cv=5,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Best Parameters:", best_params)

best_model = random_search.best_estimator_
y_pred_tuned = best_model.predict(X_test)

tuned_results = {
    'Model': 'Decision Tree (Tuned)',
    'Accuracy': accuracy_score(y_test, y_pred_tuned),
    'Precision': precision_score(y_test, y_pred_tuned),
    'Recall': recall_score(y_test, y_pred_tuned),
    'F1 Score': f1_score(y_test, y_pred_tuned)
}
print(tuned_results)


Best Parameters: {'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 10, 'criterion': 'entropy'}
{'Model': 'Decision Tree (Tuned)', 'Accuracy': 0.9946666666666667, 'Precision': 1.0, 'Recall': 0.9857142857142858, 'F1 Score': 0.9928057553956835}


In [ ]:
#Construct a Decision Tree classifier with the best parameters found
best_params = random_search.best_params_
print("Best Parameters:", best_params)

final_decision_tree = DecisionTreeClassifier(**best_params, random_state=42)

final_decision_tree.fit(X_train, y_train)

y_pred_final = final_decision_tree.predict(X_test)

final_results = {
    'Model': 'Decision Tree (Best Params)',
    'Accuracy': accuracy_score(y_test, y_pred_final),
    'Precision': precision_score(y_test, y_pred_final),
    'Recall': recall_score(y_test, y_pred_final),
    'F1 Score': f1_score(y_test, y_pred_final)
}
print("Final Decision Tree Results:", final_results)


Best Parameters: {'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 10, 'criterion': 'entropy'}
Final Decision Tree Results: {'Model': 'Decision Tree (Best Params)', 'Accuracy': 0.9946666666666667, 'Precision': 1.0, 'Recall': 0.9857142857142858, 'F1 Score': 0.9928057553956835}


In [ ]:
#compare default decisoin tree model and tuned decision tree model
default_results = {
    'Model': 'Decision Tree (Default)',
    'Accuracy': 0.94,
    'Precision': 0.97,
    'Recall': 0.96,
    'F1 Score': 0.96

tuned_results = {
    'Model': 'Decision Tree (Tuned)',
    'Accuracy': accuracy_score(y_test, y_pred_final),
    'Precision': precision_score(y_test, y_pred_final),
    'Recall': recall_score(y_test, y_pred_final),
    'F1 Score': f1_score(y_test, y_pred_final)
}

comparison = [default_results, tuned_results]
for result in comparison:
    print(result)


{'Model': 'Decision Tree (Default)', 'Accuracy': 0.94, 'Precision': 0.97, 'Recall': 0.96, 'F1 Score': 0.96}
{'Model': 'Decision Tree (Tuned)', 'Accuracy': 0.9946666666666667, 'Precision': 1.0, 'Recall': 0.9857142857142858, 'F1 Score': 0.9928057553956835}


In [ ]:
# feature_importances
importances = final_decision_tree.feature_importances_
features = X_train.columns

feature_importance = sorted(zip(importances, features), reverse=True)

top_5_features = feature_importance[:5]

print("Top 5 Features:")
for importance, feature in top_5_features:
    print(f"{feature}: {importance:.4f}")


Top 5 Features:
MOUCH3M: 0.4490
INCOME: 0.2474
CONVB_Yes: 0.0611
TRAVEL_InterGalactic: 0.0554
MOUPMO: 0.0542


In [ ]:
#ensemble predictions
estimators = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42)),
    ('svm', SVC(probability=True, random_state=42))
]

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=RandomForestClassifier(random_state=42),
    cv=5
)

stacking_clf.fit(X_train, y_train)

y_pred_stack = stacking_clf.predict(X_test)

stacking_results = {
    'Model': 'Stacking Classifier',
    'Accuracy': accuracy_score(y_test, y_pred_stack),
    'Precision': precision_score(y_test, y_pred_stack),
    'Recall': recall_score(y_test, y_pred_stack),
    'F1 Score': f1_score(y_test, y_pred_stack)
}

print("Stacking Results:", stacking_results)

Stacking Results: {'Model': 'Stacking Classifier', 'Accuracy': 0.9946666666666667, 'Precision': 1.0, 'Recall': 0.9857142857142858, 'F1 Score': 0.9928057553956835}


In [ ]:
#

In [ ]:
#

In [ ]:
#END

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#


In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#